<a href="https://colab.research.google.com/github/YunSeoHwan/DNN_Study/blob/main/C3_predict_BostonHousing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **보스턴 집값 예측하기**

# **Dataset 특징 출력**

In [93]:
from sklearn.datasets import load_boston
from sklearn.datasets import fetch_california_housing

# dataset 불러옴
dataset = load_boston()

# data의 key출력
print(dataset.keys())

# data : 사용할 특징값
# target : 정답
# feature_names : 특징 이름

# ※윤리적 이유로 사용을 권장하지 않음

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename', 'data_module'])


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

# **Data Load**

In [94]:
import pandas as pd

# data만 추출
dataFrame = pd.DataFrame(dataset["data"])

# 열이름 저장
dataFrame.columns = dataset["feature_names"]

# target 저장
dataFrame["target"] = dataset["target"]

dataFrame.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


# **Modeling**

In [99]:
import torch
import torch.nn as nn

from torch.optim.adam import Adam

# define model
model = nn.Sequential(
    nn.Linear(13, 100),
    nn.ReLU(),
    nn.Linear(100, 1)
)

# data, target split
X = dataFrame.iloc[:, :13].values   # iloc, loc 차이 -> index와 col으로 구분. 추출원리는 동일
Y = dataFrame['target'].values      # values -> 해당 value를 array로 저장

batch_size = 100
learning_rate = 0.001

# 가중치 최적화
optim = Adam(model.parameters(), lr=learning_rate)

# epoch
for epoch in range(200):

    # batch loop
    for i in range(len(X) // batch_size):

        # indexing
        start = i * batch_size
        end = start + batch_size

        # tensor transform
        x = torch.FloatTensor(X[start:end])
        y = torch.FloatTensor(Y[start:end])

        # 기울기 0으로 초기화
        optim.zero_grad()

        # 예측값, loss function
        preds = model(x)
        loss = nn.MSELoss()(preds, y)
        
        # backpropogation
        loss.backward()
        
        # optimization
        optim.step()
    
    if epoch % 20 == 0:
        print(f"epoch{epoch} loss : {loss.item()}")

epoch0 loss : 10061.8115234375
epoch20 loss : 52.58968734741211
epoch40 loss : 39.1845817565918
epoch60 loss : 38.4541015625
epoch80 loss : 38.3135986328125
epoch100 loss : 38.11167907714844
epoch120 loss : 38.00950241088867
epoch140 loss : 37.9943962097168
epoch160 loss : 37.95695877075195
epoch180 loss : 37.99419403076172


# **성능 평가**

In [100]:
prediction = model(torch.FloatTensor(X[0, :13]))
real = Y[0]
print(f"prediction : {prediction.item()} real : {real}")

prediction : 25.83799171447754 real : 24.0
